In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
path="/content/drive/MyDrive/PySpark/usage_20211116.csv"

In [4]:
import pyspark
from pyspark.sql import*
from pyspark.sql.types import*
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()
Usageschema = StructType([
StructField('namespace', StringType(), True),
StructField('timestamp', StringType(), True),
StructField('kubernetes_pod_name_keyword', StringType(),True),
StructField('kubernetes_pod_cpu_usage_node_pct', DoubleType(), True),
StructField('kubernetes_container_cpu_limit_cores', DoubleType(),True),
StructField('kubernetes_container_cpu_request_cores', DoubleType(), True),
StructField('kubernetes_pod_cpu_usage_limit_pct', DoubleType(),True),
StructField('kubernetes_pod_memory_usage_bytes', DoubleType(), True),
StructField('kubernetes_pod_cpu_usage_nanocores', DoubleType(),True),
StructField('key', StringType(), True),
StructField('environment_name', StringType(), True),
StructField('to_date', IntegerType(), True)
])


In [20]:
df = spark.read.csv(path, sep=",", header=False, schema=Usageschema)

In [21]:
df = df.groupBy('environment_name','namespace','kubernetes_pod_name_keyword').mean('kubernetes_pod_cpu_usage_nanocores')
df.sort('namespace').show(truncate = False)

+------------------------+-----------------------+--------------------------------------------------+---------------------------------------+
|environment_name        |namespace              |kubernetes_pod_name_keyword                       |avg(kubernetes_pod_cpu_usage_nanocores)|
+------------------------+-----------------------+--------------------------------------------------+---------------------------------------+
|US Commercial Dev EastUS|accnext-dev            |test-msivms-core-2-6554bbf69-54c65                |6.301028320243056E8                    |
|US Commercial Dev EastUS|accnext-dev            |test-msivms-recorder-1-7dc79db9c6-pppdz           |1.067889578298611E8                    |
|US Commercial Dev EastUS|accnext-dev            |test-msivms-core-0-899f44fb7-qnh7v                |2.206920344861111E8                    |
|US Commercial Dev EastUS|accnext-dev            |test-msivms-core-1-549cc9c6c6-62jbl               |2.6563869594444445E8                   |
|US Co

avg(kubernetes_pod_cpu_usage_nanocores)

In [22]:
df1 = df.groupBy('environment_name','namespace').sum('avg(kubernetes_pod_cpu_usage_nanocores)')
df1 = df1.select(col("environment_name"),col("namespace"),col("sum(avg(kubernetes_pod_cpu_usage_nanocores))").alias('nanocores_usage'))
df1.sort('namespace').show(truncate = False)

+------------------------+------------------------+---------------------+
|environment_name        |namespace               |nanocores_usage      |
+------------------------+------------------------+---------------------+
|US Commercial Dev EastUS|accnext-dev             |1.6280044892708335E9 |
|US Commercial Dev EastUS|acre-access-reviews-dev |7995256.975694444    |
|US Commercial Dev EastUS|alphasi500-dev          |0.0                  |
|US Commercial Dev EastUS|ams-dev                 |2.923300551968012E8  |
|US Commercial Dev EastUS|analytics-services-dev  |7.015969661226164E9  |
|US Commercial Dev EastUS|apollo-dev              |1.7109496992722565E8 |
|US Commercial Dev EastUS|appsec-dev              |1.6119547243055556E7 |
|US Commercial Dev EastUS|astroupgrademanager-dev |0.0                  |
|US Commercial Dev EastUS|avo-indoor-maps-dev     |207035.94097222225   |
|US Commercial Dev EastUS|aware-streaming-dev     |4.3155790809027776E7 |
|US Commercial Dev EastUS|axs-cloud-de

In [23]:
df2 = df.groupby('environment_name').sum('avg(kubernetes_pod_cpu_usage_nanocores)')
df2 = df2.select(col("environment_name"),col("sum(avg(kubernetes_pod_cpu_usage_nanocores))").alias('total_nanocores_usage'))
df2.show()

+--------------------+---------------------+
|    environment_name|total_nanocores_usage|
+--------------------+---------------------+
|US Commercial Dev...| 4.246917308491741...|
+--------------------+---------------------+



In [24]:
df3 = df2.join(df1,['environment_name'],'inner');
df3.show()

+--------------------+---------------------+--------------------+--------------------+
|    environment_name|total_nanocores_usage|           namespace|     nanocores_usage|
+--------------------+---------------------+--------------------+--------------------+
|US Commercial Dev...| 4.246917308491741...|         mercury-dev|   5639744.680555556|
|US Commercial Dev...| 4.246917308491741...|           ngcmf-dev|1.2108927644240883E8|
|US Commercial Dev...| 4.246917308491741...|          appsec-dev|1.6119547243055556E7|
|US Commercial Dev...| 4.246917308491741...|acre-access-revie...|   7995256.975694444|
|US Commercial Dev...| 4.246917308491741...|              rwqm68|                 0.0|
|US Commercial Dev...| 4.246917308491741...|cie-interactive-t...|  1929379.9340277775|
|US Commercial Dev...| 4.246917308491741...|virtualpartner-bo...|4.2566882981597215E8|
|US Commercial Dev...| 4.246917308491741...|         ccdrive-dev|1.954223941138081E10|
|US Commercial Dev...| 4.246917308491741...

In [25]:
df4 = df3.select(col("environment_name"),col("namespace"),((col("nanocores_usage")/col("total_nanocores_usage"))*100).alias("percentage_usage"))
df4.show(truncate = False)

+------------------------+-------------------------+---------------------+
|environment_name        |namespace                |percentage_usage     |
+------------------------+-------------------------+---------------------+
|US Commercial Dev EastUS|mercury-dev              |0.0013279619712111763|
|US Commercial Dev EastUS|ngcmf-dev                |0.028512275527543228 |
|US Commercial Dev EastUS|appsec-dev               |0.003795587733913351 |
|US Commercial Dev EastUS|acre-access-reviews-dev  |0.001882602460779698 |
|US Commercial Dev EastUS|rwqm68                   |0.0                  |
|US Commercial Dev EastUS|cie-interactive-tools-dev|4.5430127169416944E-4|
|US Commercial Dev EastUS|virtualpartner-bot-dev   |0.10023007251985913  |
|US Commercial Dev EastUS|ccdrive-dev              |4.601511635817811    |
|US Commercial Dev EastUS|cgw-dev                  |0.0024227686399550833|
|US Commercial Dev EastUS|silk-dev                 |1.7271759910286876E-7|
|US Commercial Dev EastUS

In [26]:
df5 = df4.select(sum(col("percentage_usage")))
df5.show()

+---------------------+
|sum(percentage_usage)|
+---------------------+
|   100.00000000000009|
+---------------------+

